In [1]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

## Working with industry data

In [49]:
industry_target = pd.read_csv('industry_target from sba.csv')
industry_data = pd.read_csv('industry_all from census.csv')

In [51]:
industry_target.head()

,Year,State,firm type,NAICS,Total,<20,20-99,100-499,500+,<500
0,2011.0,Alabama,firm,113,591,547,38,1,5,586
1,2011.0,Alabama,firm,114,22,21,1,NaN,NaN,22
2,2011.0,Alabama,firm,115,170,148,10,5,7,163
3,2011.0,Alabama,firm,211,31,17,1,NaN,13,18
4,2011.0,Alabama,firm,212,93,54,19,5,15,78


In [52]:
industry_data.head()

,STATE,NAICS,ENTRSIZE,INIT_ESTB,INIT_EMPL,INIT_EMPLFL_N,INIT_EMPLFL_R,NETCHG_ESTB,NETCHG_EMPL,NETCHG_EMPLFL_N,...,PCTCHG_ESTB,PCTCHG_EMPL,PCTCHG_BIRTHS_EMPL,PCTCHG_DEATHS_EMPL,PCTCHG_BIRTHS_EXP_EMPL,PCTCHG_DEATHS_CONTR_EMPL,STATEDSCR,NCSDSCR,entrsizedscr,year
0,0,--,1,6795201,121048926,G,NaN,76973,3105963,G,...,1.1,2.6,4.5,-3.7,13.6,-11.0,United States,Total,1: Total,2015
1,0,--,2,2874385,5935512,G,NaN,44862,746982,G,...,1.6,12.6,12.9,-11.6,30.7,-18.1,United States,Total,2: 1-4,2015
2,0,--,3,1009706,6566106,G,NaN,5289,174488,G,...,0.5,2.7,6.3,-5.7,18.1,-15.4,United States,Total,3: 5-9,2015
3,0,--,4,640597,8171877,G,NaN,690,171523,G,...,0.1,2.1,5.1,-4.8,15.1,-13.0,United States,Total,4: 10-19,2015
4,0,--,5,689685,20116382,G,NaN,0,410013,G,...,0.0,2.0,4.4,-4.0,12.9,-10.8,United States,Total,5: 20-99,2015


In [8]:
data = pd.read_csv('all_year_with_target.csv')
data.describe()

,year,Gross domestic product (GDP) by state,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,pce Clothing and footwear,pce Food and beverages purchased for off-premises consumption,pce Furnishings and durable household equipment,pce Gasoline and other energy goods,pce Motor vehicles and parts,...,State governments,Equals: Disposable personal income,Personal current taxes,contract_establish,end_establish,expand_establish,net_change,open_establish,total_contract,total_expand
count,1017.000000,1.017000e+03,966.000000,1017.000000,1.017000e+03,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,...,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03
mean,2006.525074,2.646932e+05,46974.765010,97.358486,2.761757e+05,981.526057,2432.219272,822.655851,1068.108161,1324.824975,...,5.103276e+06,1.971238e+08,2.646635e+07,9.280420e+05,2.268305e+05,9.536988e+05,3.998977e+04,2.411634e+05,1.154872e+06,1.194862e+06
std,5.758904,3.349438e+05,17064.875619,11.964398,3.357822e+05,198.589474,506.358814,168.080946,497.768275,226.025430,...,8.865942e+06,2.430566e+08,3.653654e+07,1.028917e+06,2.948181e+05,1.074905e+06,1.685001e+05,3.040317e+05,1.316920e+06,1.373351e+06
min,1997.000000,1.476000e+04,28368.000000,61.365000,1.952000e+04,603.000000,1483.000000,395.000000,319.000000,662.000000,...,0.000000e+00,1.025367e+07,1.392771e+06,1.092880e+05,2.017800e+04,1.098140e+05,-1.465756e+06,2.167600e+04,1.320520e+05,1.359560e+05
25%,2002.000000,6.409800e+04,38857.000000,89.874000,6.729400e+04,848.000000,2023.000000,707.000000,707.000000,1181.000000,...,9.587070e+05,4.729874e+07,5.450677e+06,2.546360e+05,5.937600e+04,2.599600e+05,1.152000e+03,6.572400e+04,3.152760e+05,3.233280e+05
50%,2007.000000,1.610450e+05,43732.000000,100.000000,1.696640e+05,941.000000,2417.000000,812.000000,981.000000,1322.000000,...,2.462547e+06,1.214731e+08,1.421030e+07,6.313720e+05,1.328940e+05,6.416600e+05,2.326600e+04,1.434660e+05,7.583500e+05,7.822120e+05
75%,2012.000000,3.203810e+05,50294.500000,104.570000,3.487080e+05,1075.000000,2720.000000,929.000000,1311.000000,1460.000000,...,6.578645e+06,2.423695e+08,3.295209e+07,1.125474e+06,2.581080e+05,1.150120e+06,8.233000e+04,2.728580e+05,1.377686e+06,1.418240e+06
max,2016.000000,2.656080e+06,172523.000000,158.916000,2.333841e+06,1888.000000,4272.000000,1486.000000,4098.000000,2489.000000,...,8.548814e+07,1.894449e+09,3.182417e+08,7.231216e+06,2.167120e+06,7.187654e+06,1.077746e+06,2.450032e+06,9.356028e+06,9.430774e+06


In [3]:
data.head()

,GeoName,year,Compensation of employees,Gross domestic product (GDP) by state,Gross operating surplus,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,Subsidies,Taxes on production and imports,...,Equals: Disposable personal income,Personal current taxes,state,contract_establish,end_establish,expand_establish,net_change,open_establish,total_contract,total_expand
0,Alabama,1997,61240,104309.0,36814,31465.0,81.042,137437.0,-328,6582,...,83209782.0,10612417.0,Alabama,648756,138914,654734,41822,174758,787670,829492
1,Alaska,1997,12205,25763.0,11234,62577.0,75.935,38358.0,-62,2386,...,15203610.0,1844364.0,Alaska,158148,37718,168904,15276,42238,195866,211142
2,Arizona,1997,69886,132795.0,54249,34485.0,67.195,163353.0,-383,9044,...,94533284.0,12903928.0,Arizona,833670,228694,1013832,198484,247016,1062364,1260848
3,Arkansas,1997,32852,60248.0,23415,30470.0,79.178,79256.0,-298,4279,...,46699786.0,5709269.0,Arkansas,371418,86722,412804,49658,94994,458140,507798
4,California,1997,574394,1082051.0,441963,41314.0,70.134,1342133.0,-5725,71419,...,761138752.0,120765010.0,California,6281922,2167120,7002930,605040,2051152,8449042,9054082


## Running General Regression

In [43]:
# using expand_establish as the target
# 'Other medical care benefits', 'Other unemployment compensation',
data = data.dropna()
Y = data['expand_establish']
X = data.drop(['expand_establish', 'state', 'GeoName', 'year', 'Compensation of employees',
              'Gross operating surplus', 'Subsidies', 'Taxes on production and imports', 
               'Taxes on production and imports less subsidies', 'Value of inventory change', 
               'Unemployment compensation for railroad employees', 'Other assistance to veterans 1',
               'Corn', 'Oats', 'Other grains', 'Soybeans', 'Forest and maple products', 'Fruits and nuts',
               'Total grains', 'Plus: Statistical adjustment', 'Plus: Value of inventory change',
               'Value of inventory change: crops', 'Value of inventory change: livestock', 
               'Value of inventory change: materials and supplies'
              ], axis=1)

In [44]:
# inout issues with compensations of employees, gross operating surplus, subsidies, taxes on production/less subsidies
X = X.iloc[:, 10:15]

In [45]:
# normalizing the dataset
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X_normalized = pd.DataFrame(np_scaled)
X_normalized.head()

,0,1,2,3,4
0,0.055135,0.028792,0.103990,0.039216,0.026410
1,0.135476,0.322331,0.186893,0.166350,0.317987
2,0.130435,0.155899,0.173868,0.059140,0.191649
3,0.013548,0.018258,0.098822,0.014864,0.006424
4,0.016698,0.278792,0.138309,0.135990,0.161670


In [46]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       expand_establish   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     149.1
Date:                Mon, 12 Nov 2018   Prob (F-statistic):          3.37e-117
Time:                        18:49:27   Log-Likelihood:                -14789.
No. Observations:                 966   AIC:                         2.959e+04
Df Residuals:                     961   BIC:                         2.961e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
pce   Other nondurable goods           -308.8613     85.912     -3.595      0.000    -477.457    -140.265
pce   Recreational goods and vehicles   112.8053    259.709      0.434      0.664    -396.857     622.467
pce  Durable goods                       73.3681     93.271      0.787      0.432    -109.670     256.406
pce  Financial services and insurance   530.7096    149.958      3.539      0.000     236.427     824.992
pce  Food services and accommodations    58.0950    143.595      0.405      0.686    -223.702     339.892
==============================================================================
Omnibus:                      526.568   Durbin-Watson:                   2.085
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3874.971
Skew:                           2.437   Prob(JB):                         0.00
Kurtosis:                      11.515   Cond. No.                         41.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
#using sklearn instead of stats models
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)

In [26]:
predictions = lm.predict(X)
#print(predictions)[0:5]

In [27]:
lm.score(X,Y)

0.973610736685525

In [28]:
lm.coef_

array([  -4.20234235,   -1.85323604, -193.27476463,    7.33224573,
       -397.25719891,   53.69246977,  118.42229521,  -37.88287215,
        154.93517445,  -38.94544452])

### What if we use open stablish?

In [33]:
Y = data['open_establish']
X = data.drop(['open_establish', 'state', 'GeoName', 'year', 'Compensation of employees',
              'Gross operating surplus', 'Subsidies', 'Taxes on production and imports', 
               'Taxes on production and imports less subsidies', 'Value of inventory change', 
               'Unemployment compensation for railroad employees', 'Other assistance to veterans 1',
               'Corn', 'Oats', 'Other grains', 'Soybeans', 'Forest and maple products', 'Fruits and nuts',
               'Total grains', 'Plus: Statistical adjustment', 'Plus: Value of inventory change',
               'Value of inventory change: crops', 'Value of inventory change: livestock', 
               'Value of inventory change: materials and supplies'
              ], axis=1)

In [34]:
# inout issues with compensations of employees, gross operating surplus, subsidies, taxes on production/less subsidies
X = X.iloc[:, 0:20]

In [31]:
# normalizing the dataset
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X_normalized = pd.DataFrame(np_scaled)
X_normalized.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.036644,0.021484,0.201710,0.053911,0.165877,0.106305,0.104491,0.063244,0.250684,0.081818,0.055135,0.028792,0.103990,0.039216,0.026410,0.090081,0.040434,0.030406,0.055742,0.033935
1,0.004502,0.237307,0.149358,0.008613,0.185624,0.331378,0.178735,0.116962,0.149973,0.180682,0.135476,0.322331,0.186893,0.166350,0.317987,0.083558,0.168422,0.200559,0.192055,0.242739
2,0.048300,0.042433,0.059764,0.065760,0.127962,0.085411,0.229148,0.079915,0.251232,0.104545,0.130435,0.155899,0.173868,0.059140,0.191649,0.038012,0.085567,0.156045,0.080323,0.050290
3,0.018614,0.014582,0.182602,0.027311,0.142970,0.012830,0.099908,0.082032,0.213465,0.153409,0.013548,0.018258,0.098822,0.014864,0.006424,0.051957,0.017788,0.036487,0.013035,0.025527
4,0.436739,0.089806,0.089891,0.604693,0.267773,0.073680,0.131072,0.034665,0.110016,0.125000,0.016698,0.278792,0.138309,0.135990,0.161670,0.092555,0.150949,0.278399,0.032278,0.092327


In [35]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         open_establish   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     1183.
Date:                Mon, 12 Nov 2018   Prob (F-statistic):               0.00
Time:                        18:40:09   Log-Likelihood:                -12232.
No. Observations:                 966   AIC:                         2.450e+04
Df Residuals:                     946   BIC:                         2.460e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
Gross domestic product (GDP) by state                              -1.9000      0.066    -28.863      0.000      -2.029      -1.771
Per capita real GDP by state                                       -0.1639      0.299     -0.549      0.583      -0.750       0.422
Quantity indexes for real GDP by state                            398.2577    310.643      1.282      0.200    -211.372    1007.887
Real GDP by state                                                   2.7067      0.065     41.637      0.000       2.579       2.834
pce   Clothing and footwear                                     -4902.5577   3989.180     -1.229      0.219   -1.27e+04    2926.108
pce   Food and beverages purchased for off-premises consumption -4789.5880   3988.697     -1.201      0.230   -1.26e+04    3038.129
pce   Furnishings and durable household equipment               -2495.0743   3843.059     -0.649      0.516      -1e+04    5046.832
pce   Gasoline and other energy goods                           -4825.4300   3988.248     -1.210      0.227   -1.27e+04    3001.405
pce   Motor vehicles and parts                                  -2646.0310   3842.758     -0.689      0.491   -1.02e+04    4895.284
pce   Other durable goods                                       -2535.9738   3842.351     -0.660      0.509   -1.01e+04    5004.543
pce   Other nondurable goods                                    -4851.7022   3988.265     -1.216      0.224   -1.27e+04    2975.168
pce   Recreational goods and vehicles                           -2645.8284   3841.285     -0.689      0.491   -1.02e+04    4892.597
pce  Durable goods                                               2637.9072   3841.791      0.687      0.492   -4901.510    1.02e+04
pce  Financial services and insurance                             -39.1024     23.731     -1.648      0.100     -85.673       7.468
pce  Food services and accommodations                             147.2254     15.496      9.501      0.000     116.815     177.636
pce  Health care                                                   35.7466     11.387      3.139      0.002      13.401      58.093
pce  Household consumption expenditures (for services)            -36.6894      9.722     -3.774      0.000     -55.768     -17.611
pce  Housing and utilities                                         52.9618     13.171      4.021      0.000      27.115      78.809
pce  Nondurable goods                                            4811.8633   3988.299      1.206      0.228   -3015.073    1.26e+04
pce  Other services                                                 8.7125     12.866      0.677      0.498     -16.537      33.962
===============================================